<a href="https://colab.research.google.com/github/karumugamio/NLAProjEnglishtoSimpleEnglishMT/blob/master/SMT_English_to_Simple_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Opening files from Google Drive and changing current working directory to Data folder 
import os
from google.colab import drive
drive.mount('/gdrive')
os.chdir('/gdrive/My Drive/NLAProjectWS')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
# if lm package import fails please update the NLTK package by !pip install nltk --upgrade to version 3.5
!pip install nltk --upgrade

     |████████████████████████████████| 1.4MB 44.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=7913dc72e004a7dfbd600d87e2598e1945d228a796540b81a36efebbee9cfa15
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
# All Import Statements

import nltk

from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

from nltk.lm import MLE

from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline

from nltk.translate.ibm1 import IBMModel1
from nltk.translate.api import AlignedSent

import dill as pickle
import time
import pandas as pd
import numpy as np
import math


In [4]:

os.chdir('/gdrive/My Drive/NLAProjectWS/workspace')
print("NLTK version used in this note book is ",nltk.__version__)
os.listdir()
NormalEngData = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.unsimplified'
SimpleEngData = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.simple'

SentencePairFile = 'sentence pairs.pickle'
IBMModelFile = 'IBMModel.pickle'
LM_EN_File = 'en_model_trigram.pickle'
LM_SI_File = 'si_model_trigram.pickle'

NLTK version used in this note book is  3.5


In [5]:
# uncomment only when you have trouble with nltk resources. This all download everything
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [0]:
allwords_en = open(NormalEngData).read()
allwords_si = open(SimpleEngData).read()
en_words = allwords_en.lower().split()
en_words.append("NULL")
en_words = set (en_words)
si_words = allwords_si.lower().split()
si_words.append("NULL")
si_words = set (si_words)

In [7]:
print("Number of Unique English Words : ",len(en_words))
print("Number of Unique Simple English Words : ",len(si_words))

Number of Unique English Words :  117830
Number of Unique Simple English Words :  107091


## Create Aligned Sentence Pairs

In [0]:
pairs = []
with open(NormalEngData) as normalLines, open(SimpleEngData) as simpleLines: 
    for x, y in zip(normalLines, simpleLines):
        x = x.strip()
        y = y.strip()
        #print("{0}\t{1}".format(x, y))
        pair = (x,y)
        pairs.append(pair)
totalPairs = len(pairs)

## Split Entire Data Corpus into Train Test partition

In [9]:
splitRatio = 0.7
cutoff = round(totalPairs*splitRatio)
print(type(pairs))
train = pairs[0:int(cutoff)]
test = pairs[int(cutoff)+1:int(totalPairs)-1]
print("Length of Total Pairs created is {}".format(totalPairs))
print("Length of train Pairs created is {}".format(len(train)))
print("Length of test Pairs created is {}".format(len(test)))

<class 'list'>
Length of Total Pairs created is 137362
Length of train Pairs created is 96153
Length of test Pairs created is 41207


##Creating Language Models

In [0]:
en_tokenized_text = []
si_tokenized_text = []
for p in pairs:
    en_tokenized_text.append(nltk.word_tokenize(p[0]))
    si_tokenized_text.append(nltk.word_tokenize(p[1]))

In [0]:
# Using Tri Gram Model for this SMT Problem Statement hence N =3 for other NGrams Please replace n with other values
n = 3
en_model = MLE(n)
en_train_data, en_padded_sents = padded_everygram_pipeline(n, en_tokenized_text)
en_model.fit(en_train_data, en_padded_sents)

si_model = MLE(n)
si_train_data, si_padded_sents = padded_everygram_pipeline(n, si_tokenized_text)
si_model.fit(si_train_data, si_padded_sents)


In [0]:
# Simplifying read and preprocess for creating models
def read_sents(filename):
    sents = []
    c=0
    with open(filename,'r') as fi:
        for li in fi:
            sents.append(li.split())
    return sents

In [13]:
# Arrived Max Count as 96153 by 0.7 times the available corpus.
# We are using IBM Model 1 for creating Translation Probablity
max_count = 96153
eng_sents_all = read_sents(NormalEngData)
fr_sents_all = read_sents(SimpleEngData)
eng_sents = eng_sents_all[:max_count]
fr_sents = fr_sents_all[:max_count]
print("Size of english sentences: ", len(eng_sents))
print("Size of french sentences: ", len(fr_sents))
aligned_text = []
for i in range(len(eng_sents)):
    al_sent = AlignedSent(fr_sents[i],eng_sents[i])
    aligned_text.append(al_sent)
print("Training IBM model 1")
start = time.time()
ibm_model = IBMModel1(aligned_text,5)

end = time.time()
executionTime = end - start
print("Training complete, Time taken for training model is :",executionTime)

Size of english sentences:  96153
Size of french sentences:  96153
Training smt model
Training complete, Time taken for training model is : 444.95045161247253


In [0]:
# Its important to use binary mode 
dbfile = open(IBMModelFile, 'ab') 
# source, destination 
pickle.dump(ibm_model, dbfile)                      
dbfile.close() 

In [0]:
#si_model
# Its important to use binary mode 
dbfile1 = open(LM_SI_File, 'ab') 
# source, destination 
pickle.dump(si_model, dbfile1)                      
dbfile1.close() 

In [0]:
#si_model
# Its important to use binary mode 
dbfile2 = open(LM_EN_File, 'ab') 
# source, destination 
pickle.dump(en_model, dbfile2)                      
dbfile2.close() 

In [0]:
# Its important to use binary mode 
dbfile3 = open(SentencePairFile, 'ab') 
# source, destination 
pickle.dump(pairs, dbfile3)                      
dbfile3.close() 

In [0]:
#Loading all Pickle Files for Creating actual SMT using HMM- Viterbi Algorithm
# Source Tutorial for Viterbi- https://youtu.be/ECu_KQV3V30
pickle_in = open(SentencePairFile,"rb")
pairs = pickle.load(pickle_in)
pickle_in = open(LM_EN_File,"rb")
en_model = pickle.load(pickle_in)
pickle_in = open(LM_SI_File,"rb")
si_model = pickle.load(pickle_in)
pickle_in = open(IBMModelFile,"rb")
ibm_model = pickle.load(pickle_in)

In [0]:
def translate(inputSentence):
  inputSentence = inputSentence.split()

  pi = defaultdict(lambda:float(1))
  bp = defaultdict(lambda:"a")

  for k in range(0,len(inputSentence)):
    for eachword in si_words:
      for(w1,w2) in si_model.something(eachword) :
        pi_now = pi[k-1]*si_model.score(eachword,(w1,w2)) * "translation probablity from IBM model for kth given normal word vs each word in english"
        
        if (pi[k]==1 or pi[k]<pi_now):
          pi[k] = pi_now
          bp[k+1] = eachword
  return bp